In [7]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd

In [8]:
def get_eod_data(start_date, end_date, tickers):
    """
    Get EOD data for a list of tickers
    """    
    eod_df = pd.DataFrame([])

    for t in tickers:
        results = pdr.DataReader(t, 'yahoo', start_date, end_date)
        results['ticker'] = t  
        eod_df = eod_df.append(results)
        
    return eod_df

In [9]:
def compute_analytics(df, indicators):
    
    """ 
    Get calcs for each ticker in the df
    """
    
    data = None
    
    for indicator in indicators:

        # Using eval so we can iterate over a list of indicators 
        data = eval('TA.' + indicator + '(df)')
        
        # TA returns a set -> so we convert that to a data frame
        data = data.to_frame()    

        # The columns names are too long so I split the string 
        # and join that with the indicator name for my indicator column name 
        first_string = data.columns[0].split()[0]
        data = data.rename(columns={ data.columns[0]: first_string })    
        data = data.add_prefix(indicator + '_')

        # Merge data frames based on the date
        df = df.merge(data, left_index=True, right_index=True)
        
    return df

In [10]:
if __name__ == '__main__':
    
    start = datetime.now().date() - timedelta(days=20)
    end = datetime.now().date()
    ticker_list = ['AAPL', 'TSLA']
    eod_data = get_eod_data(start, end, ticker_list)
    
    indicator_list = ['SMA', 'EMA']
    analytics_data = eod_data.groupby(['ticker']).apply(lambda x: compute_analytics(x, indicator_list)).reset_index()

In [11]:
# Update the column names to follow PEP8
analytics_data = analytics_data.rename(columns={'Date':'date', 'High': 'high', 'Low': 'low', 'Open': 'open', 
                                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                               'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})

In [12]:
analytics_data.loc[analytics_data['ticker'] == 'AAPL']

,date,high,low,open,close,vol,adj_close,ticker,sma_41,ema_9
0,2020-10-26,116.550003,112.879997,114.010002,115.050003,111850700.0,114.851852,AAPL,NaN,115.050003
2,2020-10-27,117.279999,114.540001,115.489998,116.599998,92276800.0,116.399178,AAPL,NaN,115.911112
4,2020-10-28,115.430000,111.099998,115.050003,111.199997,143937800.0,111.008476,AAPL,NaN,113.980327
6,2020-10-29,116.930000,112.199997,112.370003,115.320000,146129200.0,115.121384,AAPL,NaN,114.434146
8,2020-10-30,111.989998,107.720001,111.059998,108.860001,190272600.0,108.672516,AAPL,NaN,112.775963
10,2020-11-02,110.680000,107.320000,109.110001,108.769997,122866900.0,108.582664,AAPL,NaN,111.690124
12,2020-11-03,111.489998,108.730003,109.660004,110.440002,107624400.0,110.249794,AAPL,NaN,111.373751
14,2020-11-04,115.589996,112.349998,114.139999,114.949997,138235500.0,114.752022,AAPL,NaN,112.233190
16,2020-11-05,119.620003,116.870003,117.949997,119.029999,126387100.0,118.824997,AAPL,NaN,113.803287
18,2020-11-06,119.199997,116.129997,118.320000,118.690002,114457900.0,118.690002,AAPL,NaN,114.898195
